### Imports

In [1]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

import cobra
import riptide

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

print('Imports Complete')

Imports Complete


### Read transcriptomics file

In [2]:
data = pd.read_csv ('data/matrix.csv')

print(data.shape)
data.head()

(138745, 14)


,ensembl_id,ID_REF,EED-Control-1,EED-Control-2,EED-Control-3,EED-ND-1,EED-ND-2,EED-ND-3,Healthy-Control-1,Healthy-Control-2,Healthy-Control-3,Healthy-ND-1,Healthy-ND-2,Healthy-ND-3
0,NaN,AFFX-BkGr-GC03_st,2.509910,2.111326,1.777033,2.309331,1.728352,1.860814,1.867672,1.919417,1.841486,2.134630,1.974421,2.187432
1,NaN,AFFX-BkGr-GC04_st,2.295783,1.900880,1.797043,1.938862,1.621349,1.958245,1.757484,1.827596,1.828458,1.971875,1.976003,2.063035
2,NaN,AFFX-BkGr-GC05_st,2.257641,1.879776,1.755351,1.967621,1.672276,1.901808,1.839612,1.860259,1.871003,1.968703,2.036299,2.033751
3,NaN,AFFX-BkGr-GC06_st,2.323094,1.934329,1.815905,2.030437,1.698393,1.954793,1.822267,1.857776,1.872710,2.000145,2.089354,2.102391
4,NaN,AFFX-BkGr-GC07_st,2.361259,1.945320,1.874545,1.995780,1.718467,1.982185,1.852268,1.898476,1.911582,2.041037,2.106554,2.105908


In [3]:
# Create dataframes of the patients
dfs = {
    'EED': pd.DataFrame(),
    'Healthy': pd.DataFrame()
}

id_map = {
    'EED': ['EED-ND-1','EED-ND-2','EED-ND-3'],
    'Healthy': ['Healthy-Control-1','Healthy-Control-2','Healthy-Control-3']
}

data = data[data['ensembl_id'].notna()]

dfs['EED'] = data[['ensembl_id',
                   'EED-ND-1',
                   'EED-ND-2',
                   'EED-ND-3']]

dfs['Healthy'] = data[['ensembl_id',
                       'Healthy-Control-1',
                       'Healthy-Control-2',
                       'Healthy-Control-3']]

In [4]:
# Gene Map with all the genes present in Recon3D
gene_name_num = pd.read_csv('data/gene_name_number.tsv', sep='\t')
gene_name_num = gene_name_num[['gene_number', 'symbol', 'ensembl_gene']]
gene_name_num

for keys in dfs.keys():
    
    # Merge gene number annotations to dataframe
    dfs[keys] = pd.merge(gene_name_num, dfs[keys], left_on='ensembl_gene', right_on='ensembl_id')

In [5]:
print(dfs['EED'].shape)
print(dfs['Healthy'].shape)

(3585, 7)
(3585, 7)


### Run Riptide for all patients

In [6]:
# Load Model
model = cobra.io.load_matlab_model('data/Recon3D_301.mat')
cobra.util.array.create_stoichiometric_matrix(model).shape

def run_riptide(fname, pat_name):
    
    transcript_abundances = riptide.read_transcription_file(fname,  norm=False)
    riptide_object = riptide.contextualize(model=model, transcriptome=transcript_abundances, fraction=1.)

    riptide_fva = riptide_object.flux_samples
    riptide_fva['sample_number'] = pat_name
    
    return riptide_fva

No defined compartments in model Recon3D. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, i, l, m, n, r, x


In [7]:
# Run for each patient
for keys in dfs.keys():
    print('Running for - {}'.format(keys))
    for pat_id in id_map[keys]:
        dfs[keys][['gene_number', pat_id]].to_csv('sample.tsv', header=False, index=False, sep='\t')
        pat_output = run_riptide('sample.tsv', pat_id)
        pat_output.to_csv('output/{}.csv'.format(pat_id), index=False)
        
# Combine flux samples  
for keys in dfs.keys():
    list_of_pat_flux = []
    for pat_id in id_map[keys]:
        list_of_pat_flux.append(pd.read_csv('output/{}.csv'.format(pat_id)))
    pd.concat(list_of_pat_flux, join='inner', ignore_index=0).to_csv(\
    'output/riptide_{}_flux_sample.csv'.format(keys), index=False) 

Running for - EED

Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 818 from 13543 (93.96% change)
Metabolites pruned to 767 from 8399 (90.87% change)
Flux through the objective DECREASED to ~995.1377 from ~1000.0 (0.49% change)
Context-specific metabolism correlates with transcriptome (r=-0.18, p<0.001 *)

RIPTiDe completed in, 3 minutes and 6 seconds 


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 1173 from 13543 (91.34% change)
Metabolites pruned to 1185 from 8399 (85.89% change)
Flux through the objective DECREASED to ~992.8036 from ~1000.0 (0.72% change)
Context-specific metabolism correlates with transcriptome (r=-0.081, p=0.006 *)

RIPTiDe completed in, 3 minutes and 21 seconds 


Initializing model and integrating transcriptomic data...
Pruning zero flux

### Run ML model on aggregated flux files

In [8]:
df_eed = pd.read_csv('output/riptide_EED_flux_sample.csv')
df_healthy = pd.read_csv('output/riptide_Healthy_flux_sample.csv')

df_eed['label'] = 1
df_healthy['label'] = 0

df = pd.concat([df_eed, df_healthy], join='inner')
df = df.reset_index(drop=True)

#### Model instructions

- Running the model 90 times with 2 patients (1 healthy + 1 EED) in test and 4 patients (2 healthy + 2 EED) in train.
- The test patients are each randomly selected in every 3rd iteration and the next patient is taken in the next two iterations.
- Only keeping top reactions from the runs where gains are greater than 0.2. This rule can be modified based on project goals.

In [9]:
imp_list = []
accuracy_tracker = []
f1_tracker = []
precision_tracker = []
recall_tracker = []

val_distribution_tracker = []
imp_df = []

i = 0
x = 0
y = 0

# Run model 90 times
for _ in tqdm(range(90)):
    
    # Select the test sample
    if (i%3 == 0):
        x = random.randint(0, 2)
        y = random.randint(0, 2)
        i += 1
        print(i,x,y)
    else:
        x = (x+1)%3
        y = (y+1)%3
        i += 1
        print(i,x,y)
    
    test_sample = [id_map['EED'][x], id_map['Healthy'][y]]
    train_sample = list(set(list(df_eed['sample_number'].unique()) +\
                            list(df_healthy['sample_number'].unique())) - set(test_sample))
    
    X_train, y_train = df.loc[df['sample_number'].isin(train_sample), list(set(df.columns)-set(['sample_number', 'label']))].reset_index(drop=True),\
    df.loc[df['sample_number'].isin(train_sample), ['label']].reset_index(drop=True)

    X_test, y_test = df.loc[df['sample_number'].isin(test_sample), list(set(df.columns)-set(['sample_number', 'label']))].reset_index(drop=True),\
    df.loc[df['sample_number'].isin(test_sample), ['label']].reset_index(drop=True)

    y_train = y_train['label']
    y_test = y_test['label']
    
    # Train random forest
    rf = RandomForestClassifier(n_estimators=100)
    rf.fit(X_train, y_train)

    y_train_pred = rf.predict(X_train)
    print('Training accuracy on selected features: %.3f' % accuracy_score(y_train, y_train_pred))    

    y_test_pred = rf.predict(X_test)    
    print('Testing accuracy on selected features: %.3f' % accuracy_score(y_test, y_test_pred))    
    print('F1 Score: {}'.format(f1_score(y_test, y_test_pred)))
    print('Distribution: ', max(y_test.mean(), (1-y_test.mean())))
    
    selected_cols = X_train.columns[np.argsort(rf.feature_importances_)[::-1]]
    
    val_distribution_tracker.append(y_test.mean())
    f1_tracker.append(f1_score(y_test, y_test_pred))
    precision_tracker.append(precision_score(y_test, y_test_pred))    
    recall_tracker.append(recall_score(y_test, y_test_pred))        
    accuracy_tracker.append(accuracy_score(y_test, y_test_pred))
    gain = accuracy_score(y_test, y_test_pred)-max(y_test.mean(), (1-y_test.mean()))
    
    if gain>0.2:
        imp_list += list(selected_cols[:50])
        imp_df.append(pd.DataFrame({'reactions': selected_cols[:50], 'rank': range(1, 51)}))
        print('Imp features: ', selected_cols[:10])
        print('-'*20)
        print('Gain: ', gain)
        print('-'*20)
    else:
        print('Failed to gain: ', gain)
        
    print('*'*20)
    print('*'*20)

  0%|          | 0/90 [00:00<?, ?it/s]

1 1 1


  1%|          | 1/90 [00:01<02:22,  1.60s/it]

Training accuracy on selected features: 1.000
Testing accuracy on selected features: 0.781
F1 Score: 0.8068935041979673
Distribution:  0.5
Imp features:  Index(['HMR_0393', 'ACACT1x', 'GLYLYSCYSt', 'HMR_0327', 'sink_tyr_L[c]',
       'NADtpu', 'NADtx', 'HMGCOAtx', 'r0463', 'HISCYSCYSt'],
      dtype='object')
--------------------
Gain:  0.2815
--------------------
********************
********************
2 2 2


  2%|▏         | 2/90 [00:03<02:17,  1.56s/it]

...

Training accuracy on selected features: 1.000
Testing accuracy on selected features: 0.011
F1 Score: 0.0
Distribution:  0.5
Failed to gain:  -0.4885
********************
********************
89 2 1


 99%|█████████▉| 89/90 [02:33<00:01,  1.78s/it]

Training accuracy on selected features: 1.000
Testing accuracy on selected features: 0.519
F1 Score: 0.6685063815108658
Distribution:  0.5
Failed to gain:  0.019499999999999962
********************
********************
90 0 2


100%|██████████| 90/90 [02:35<00:00,  1.72s/it]

Training accuracy on selected features: 1.000
Testing accuracy on selected features: 0.359
F1 Score: 0.528870908422214
Distribution:  0.5
Failed to gain:  -0.1405
********************
********************


In [10]:
print('Average Accuracy score: {}'.format(np.mean(accuracy_tracker)))
print('Average F1 score: {}'.format(np.mean(f1_tracker)))
print('Average validation distribution: {}'.format(np.mean(val_distribution_tracker)))
print('Average precisions score: {}'.format(np.mean(precision_tracker)))
print('Average recall score: {}'.format(np.mean(recall_tracker)))

Average Accuracy score: 0.43077777777777776
Average F1 score: 0.46211422351263665
Average validation distribution: 0.5
Average precisions score: 0.4214492050346831
Average recall score: 0.5605666666666668


In [11]:
# Top important reactions
average_rank = pd.DataFrame({'rank': pd.concat(imp_df).groupby('reactions')['rank'].mean()}).reset_index()
average_rank = average_rank.sort_values(by='rank').head(20).reset_index(drop=True)
average_rank.head(20)

,reactions,rank
0,HMGCOAtx,5.850000
1,r0463,6.809524
2,RTOT4,7.285714
3,ACACT1x,8.047619
4,GLYLYSCYSr,8.166667
5,GLYLYSCYSt,8.500000
6,sink_tyr_L[c],12.650000
7,NADtx,13.095238
8,NADtpu,13.142857
9,CATp,13.285714


In [12]:
pd.DataFrame(imp_list)[0].value_counts().head(20)

RTOT4            21
ACACT1x          21
r1428            21
C140CPT1         21
NADtpu           21
NADtx            21
CATp             21
r0463            21
r0202            20
HMR_0393         20
HMR_0327         20
HMGCOAtx         20
sink_tyr_L[c]    20
EX_pcrn[e]       19
sink_arg_L[c]    18
GLUt2m           17
RE0578C          16
EX_ttdcrn[e]     15
r0181            14
TTDCRNNAt        13
Name: 0, dtype: int64